## Guided Project: Answering Business Questions Using SQL
---

This project we are going to work with `Chinook` data base to unswer some busines questions. See chart below to find out tabes involved in deceision-making.

Let's start with loading data base to working enviroment.

![Chinook_DB](Chinook_DB.jpg)

In [19]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Shortly familiarize ourself with tables and views within database (DB)

In [20]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");


 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# 1. Busines task - finding most popular genres

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

* Regal - Hip-Hop
* Red Tone - Punk
* Meteor and the Girls - Pop
* Slim Jim Bites - Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

You'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.

To answer this question I had a closer look into dataset and encircled tables I will use. Those are:
1. `invoice line` - which is table of the main concern since it gather invoice_id for each track. That will help answer question `how often track was purchased`

*tables to follow will add info to answer questions regarding genre and custoers location*

2. `invoice` and `customer`- to find customer's nationality (i.e. Countru customer from - we are interested in USA only)
3. `track` and `genre` tables to find out genre name for each track.

I will use `WITH` clause to prepare table to work with.

In [21]:
%%sql
WITH track_purchased_USA AS 
    (SELECT
        il.track_id,
        COUNT(il.invoice_line_id) times_track_purchsed,
        c.country
    FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    WHERE c.country = 'USA'
    GROUP BY il.track_id),
     track_genre AS
    (SELECT
        t.track_id,
        g.name
    FROM track t
    INNER JOIN genre g ON t.genre_id = g.genre_id
    GROUP BY t.track_id)


SELECT 
    tg.name genre,
    SUM(t_USA.times_track_purchsed) tracks_purchased,
    ROUND(100*CAST(SUM(t_USA.times_track_purchsed) AS Float)/(SELECT SUM(times_track_purchsed)
                                    FROM track_purchased_USA),2) percentage
    FROM track_genre tg
    INNER JOIN track_purchased_USA t_USA ON t_USA.track_id = tg.track_id
    GROUP BY genre
    ORDER BY tracks_purchased DESC
    LIMIT 10;   


 * sqlite:///chinook.db
Done.


genre,tracks_purchased,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Having table above we can say that `Hip-Hop` is not a good choice for album to be added to the store. `Hip-Hop` is only 9th on the list. While `Punk` is 2nd, `Blues` is 5th and `Pop` is 7th. Those three will apparently attract more buyers. However we can see that gap between `pop` and `hip-hop` is just `0,19` percens or two albums in absolute figures which is neglectible. 

We've got to be carefull while making deceision. I beleive we have to consider each albums qualities to make final deceision between `pop` and `hip-hop` albums. 

# 2. Busines Task - employees survey

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. I have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

You might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.

In [22]:
%%sql
WITH customers_per_agent AS
    (SELECT
        e.employee_id,
        e.last_name||' '||e.first_name sales_agent,
        e.hire_date,
        COUNT(c.customer_id) customers
    FROM employee e
    INNER JOIN customer c ON c.support_rep_id = e.employee_id
    WHERE e.title = 'Sales Support Agent'
    GROUP BY 1),

     customer_purchases AS
    (SELECT
        c.support_rep_id,
        COUNT(i.invoice_id) purchases,
        ROUND(SUM(i.total),2) total_paid,
        ROUND(SUM(i.total)/COUNT(i.invoice_id),2) avg_purchase_price
    FROM customer c
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY 1)

SELECT
    customers_per_agent.sales_agent,
    customers_per_agent.hire_date,
    customers_per_agent.customers,
    customer_purchases.purchases,
    customer_purchases.total_paid,
    customer_purchases.avg_purchase_price
FROM customers_per_agent
INNER JOIN customer_purchases ON customer_purchases.support_rep_id = customers_per_agent.employee_id
ORDER BY 5 DESC;



 * sqlite:///chinook.db
Done.


sales_agent,hire_date,customers,purchases,total_paid,avg_purchase_price
Peacock Jane,2017-04-01 00:00:00,21,212,1731.51,8.17
Park Margaret,2017-05-03 00:00:00,20,214,1584.0,7.4
Johnson Steve,2017-10-17 00:00:00,18,188,1393.92,7.41


Table above sorted by `total_paid` column which stands for amount of money each sales agent brought co employer. Taking into account `hire_date` I can conclude that in average all sales agents are on the same level. The only significant difference is that Jane's Peacock avg_price per purchase is bigger. I would attrebute this to the fact that she works from the earlier stage, when customers was purchsing more tracks at once, since there was nothing in their Playlists.

Margaret Park managed to fix more deals(purchases) than Jane who works month longer. 

Steve Johnson is working 6 month less than Jane. His stats are in line with others. 

In general sales agents perform equally good. There is positive correlation between all stats and employemnent duration.

# 3. Busines task - Analyzing Sales by Country

Your next task is to analyze the sales data for customers from each different country. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

    total number of customers
    total value of sales
    average value of sales per customer
    average order value

Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis. You can use the following 'trick' to force the ordering of "Other" to last in your analysis.

If there is a particular value that you would like to force to the top or bottom of results, you can put what would normally be your most outer query in a subquery with a case statement that adds a numeric column, and then in the outer query sort by that column. Here's an example - let's start by creating a view so we're working with a manageable number of rows:


In [23]:
%%sql
WITH customers_per_country AS
    (SELECT
        c.country,
        COUNT(DISTINCT(c.customer_id)) customers,
        ROUND(SUM(i.total),2) total_sales,
        ROUND(CAST(SUM(i.total) AS Float)/COUNT(DISTINCT(c.customer_id)),2) avg_sale_per_customer,
        ROUND(AVG(i.total),2) avg_order_value
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    GROUP BY 1
    ORDER BY 3 DESC),

      single_customer_countries AS 
    (SELECT
        'Others' country,
        SUM(customers) customers,
        ROUND(SUM(total_sales),2) total_sales,
        AVG(avg_sale_per_customer) avg_sale_per_customer,
        ROUND(AVG(avg_order_value),2) avg_order_value
    FROM customers_per_country
    WHERE customers = 1),

      country_customer_stats AS 
    (SELECT * FROM customers_per_country WHERE customers <> 1
    UNION
    SELECT * FROM single_customer_countries
    ORDER BY total_sales DESC)


SELECT 
    country,
    customers,
    total_sales,
    avg_sale_per_customer,
    avg_order_value
FROM    (SELECT
            country_customer_stats.*,
            CASE
                WHEN country_customer_stats.country = 'Others' THEN 1
                ELSE 0
            END AS sort
        FROM country_customer_stats)
ORDER BY sort ASC;

 * sqlite:///chinook.db
Done.


country,customers,total_sales,avg_sale_per_customer,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Others,15,1094.94,72.996,7.44


From table above we can notice that there are more customers and consequently sales in North America. However worth saying that most European customers as well as Indian tend to spend more money. It might be a good idea to find out what is other countries users favorite genres. 

To make long story short it will be profitable to expand on other markets and adjust Chinook music to wider needs.

# 4. Business task - Albums VS Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

    purchase a whole album
    purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when you are performing an analysis to have 'edge cases' which prevent you from getting a 100% accurate answer to your question. In this instance, we have two edge cases to consider:

    Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
    
    Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

In [ ]:
%%sql
WITH track_from_invoice AS
    (SELECT
        il.invoice_id,
        il.track_id,
        t.album_id
    FROM invoice_line il 
    INNER JOIN track t ON il.track_id = t.track_id
    GROUP BY 1),

    purchase_type AS
    (SELECT
        track_from_invoice.invoice_id,
        CASE
            WHEN 
            (SELECT t.track_id FROM track t 
            WHERE t.album_id =      (SELECT album_id
                                    FROM track t2
                                    WHERE track_from_invoice.track_id = t2.track_id)
            EXCEPT
            SELECT il.track_id FROM invoice_line il 
            WHERE il.invoice_id =   (SELECT invoice_id
                                    FROM invoice_line il2
                                    WHERE track_from_invoice.invoice_id = il2.invoice_id)) IS NULL
            AND
            (SELECT il.track_id FROM invoice_line il 
            WHERE il.invoice_id =   (SELECT invoice_id
                                    FROM invoice_line il2
                                    WHERE track_from_invoice.invoice_id = il2.invoice_id)
            EXCEPT 
            SELECT t.track_id FROM track t 
            WHERE t.album_id =      (SELECT album_id
                                    FROM track t2
                                    WHERE track_from_invoice.track_id = t2.track_id)) IS NULL
            THEN 1
            ELSE 0
            END AS album_purchase
    FROM track_from_invoice)

SELECT
    COUNT(invoice_id) total_purchases,
    SUM(album_purchase) album_purchases,
    ROUND(CAST(SUM(album_purchase) AS Float)/COUNT(invoice_id),3)*100 percentage
FROM purchase_type


 * sqlite:///chinook.db
Done.


total_purchases,album_purchases,percentage
614,114,18.6


From the query above we can conclude that `18.6 %` of purchases consist of a single album. Still this is quite a figure and we cannot risk to loose fifth part of customers. This matter needs further analysis in regard of profit to be gathered from changing the buying strategy. Nevertheless, intermediate conclusion - `most purchses have track selection from different albums`.